In [1]:
"""
File name: P1.ipynb
Author: Wajid Khattak
Date created: 2017-10-10
Date last modified: 2017-10-14
Python Version: 3.5.2

"""
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    """
    Applies the Canny transform
    Returns edges
    """
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
                
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #find those bits that are both 1 i.e bits that are 1 in the canny edge output and the mask
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image,mask


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """   
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Applies Hough Transform
    `img` should be the output of a Canny transform.    
    Returns hough lines and image drawn on a blank image.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
       
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    
    return lines, line_image

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    Combines two images based on α,β,λ values
    
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def find_left_right_lines(lines):
    """
    Finds left and right lane lines by calculating slope of the lines.
    Lines with negative slopes are assigned to left side & vice-versa.
    However, lines with slope between -0.35 & +0.35 are ignored,
    as these generally represent horizontal lines e.g. shadows or some other non-lane markers picked up by Hough Transform
    """
    left_lines = []
    right_lines = []

    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if (slope < 0.35 and slope > -0.35):
                  continue
            elif (slope <= -0.35):
                  left_lines.append(line)
            else:
                right_lines.append(line)

    return left_lines,right_lines

def extrapolate_lines(left_lines, right_lines, mask_vertices):
    """
    Extrapolates lines by fitting a line based on input set of x,y points.
    
    Main idea is to use the min & max y points of the mask (the one that was used to construct the region of interest) 
    as the min, max y points of the extrapolated lines because this is the set of vertical extents that needs to be 
    covered by the extrapolated lines. Now that we know the y points, we can find the x points by x = (y-b)/m.
    
    Retunrs an array of left and right extrapolated lines.
    """
    left_lines_x_points = []
    left_lines_y_points = []
    right_lines_x_points = []
    right_lines_y_points = []

    #extract x,y points from all left lines
    for line in left_lines:
        for x1, y1, x2, y2 in line:
            left_lines_x_points += [x1, x2]
            left_lines_y_points += [y1, y2]

    #extract x,y points from all right lines
    for line in right_lines:
        for x1, y1, x2, y2 in line:
            right_lines_x_points += [x1, x2]
            right_lines_y_points += [y1, y2]

    #fit left line
    left_line_fit = np.polyfit(left_lines_x_points, left_lines_y_points, 1)
    #get m & b values
    m_left,b_left = left_line_fit
    #find max x by plugging in max y of the mask
    max_x_left = int((mask_vertices[0][0][1]-b_left)//m_left)
    #find min x by plugging in min y of the mask
    min_x_left = int(( mask_vertices[0][1][1]-b_left)//m_left)
    
    #fit right line
    right_line_fit = np.polyfit(right_lines_x_points, right_lines_y_points, 1)
    #get m & b values
    m_right,b_right = right_line_fit
    #find max x by plugging in max y of the mask
    max_x_right = int((mask_vertices[0][0][1]-b_right)//m_right)
    #find min x by plugging in min y of the mask
    min_x_right = int(( mask_vertices[0][2][1]-b_right)//m_right)
    
    extrapolated_lines = np.array([
        [[min_x_left,  mask_vertices[0][2][1], max_x_left, mask_vertices[0][0][1]]],
        [[min_x_right,  mask_vertices[0][1][1], max_x_right, mask_vertices[0][0][1]]]
    ])
    
    return extrapolated_lines

def find_lane_lines(input_image_full_path):
    """
    Finds lane lines for an input image path and writes out the processed image to 'test_images_output' directory.
    Internally it calls the 'process_image()' method
    """
    print('Finding lane lines for: ',input_image_full_path)
    image = mpimg.imread(input_image_full_path)
    output_image = process_image(image)
    
    output_image_path_prefix = 'test_images_output/'
    output_image_name = input_image_full_path.split('/')[-1]
    output_image_full_path = output_image_path_prefix + output_image_name
    #convert to RGB as cv2 uses BGR color scheme
    output_image_RGB = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    print('Writing processed image: ',output_image_full_path, '\n')
    cv2.imwrite(output_image_full_path,output_image_RGB)

def find_lane_lines_raw(input_image_full_path):
    """
    Finds raw lane lines (Hough lines) for an input image path and writes out the processed image to 'test_images_output' directory.
    Internally it calls the 'process_image_raw_lines()' method
    """
    print('Finding raw lane lines for: ',input_image_full_path)
    image = mpimg.imread(input_image_full_path)
    output_image = process_image_raw_lines(image)
    
    output_image_path_prefix = 'test_images_output/raw-'
    output_image_name = input_image_full_path.split('/')[-1]
    output_image_full_path = output_image_path_prefix + output_image_name 
    #convert to RGB as cv2 uses BGR color scheme
    output_image_RGB = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    print('Writing processed image: ',output_image_full_path, '\n')
    cv2.imwrite(output_image_full_path,output_image_RGB)    
    
def process_image(image):
    """
    Main pipeline function that calls other helper fuctions to find lane lines
    """
    #scale down image so that the region of interest mask fits in properly
    if (image.shape[0] != 540 and image.shape[1] != 960):
        image = cv2.resize(image,None,fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
        
    # To make sure we only select lane lines, filter out non-white & non-yellow pixels.
    # This is especially helpful when tree shadows & median create lines,
    # which get interpreted as part of left or right lane lines
    color_range = [(np.array([175, 175, 0], dtype = "uint8"), np.array([255, 255, 255], dtype = "uint8"))]
    mask_white_yellow = cv2.inRange(image,color_range[0][0],color_range[0][1])
    white_yellow_image = cv2.bitwise_and(image,image, mask= mask_white_yellow)
    
    #turn image to gray scale
    gray_image = grayscale(white_yellow_image)
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray_image, kernel_size)

    # Parameters for Canny edge detection 
    low_threshold = 50
    high_threshold = 115
    canny_edges = canny(blur_gray, low_threshold, high_threshold)
    
    # Define a mask so that we only get lines from our region of interest i.e. filter out non-lane line area
    imshape = image.shape
    vertices = np.array([[(60,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges,mask = region_of_interest(canny_edges, vertices)
    
    # Define the Hough transform parameters
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 18 # minimum number of pixels making up a line
    max_line_gap = 30 # maximum gap in pixels between connectable line segments
    lines,hough_line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    #combined_image = weighted_img(hough_line_image, image)
    
    #find left and right lines
    left_lines,right_lines = find_left_right_lines(lines)

    #extrapolate lines 
    extrapolated_lines = extrapolate_lines(left_lines, right_lines, vertices)
    
    #Draw extrapolated lines on the original image
    processed_image = np.copy(image)
    draw_lines(processed_image, extrapolated_lines)
    
    return processed_image

def process_image_raw_lines(image):
    """
    Main pipeline function that calls other helper fuctions to find raw lane lines
    """
    #scale down image so that the region of interest mask fits in properly
    if (image.shape[0] != 540 and image.shape[1] != 960):
        image = cv2.resize(image,None,fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
        
    # To make sure we only select lane lines, filter out non-white & non-yellow pixels.
    # This is especially helpful when tree shadows & median create lines,
    # which get interpreted as part of left or right lane lines
    color_range = [(np.array([175, 175, 0], dtype = "uint8"), np.array([255, 255, 255], dtype = "uint8"))]
    mask_white_yellow = cv2.inRange(image,color_range[0][0],color_range[0][1])
    white_yellow_image = cv2.bitwise_and(image,image, mask= mask_white_yellow)
    
    #turn image to gray scale
    gray_image = grayscale(white_yellow_image)
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray_image, kernel_size)

    # Parameters for Canny edge detection 
    low_threshold = 50
    high_threshold = 115
    canny_edges = canny(blur_gray, low_threshold, high_threshold)
    
    # Define a mask so that we only get lines from our region of interest i.e. filter out non-lane line area
    imshape = image.shape
    vertices = np.array([[(60,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges,mask = region_of_interest(canny_edges, vertices)
    
    # Define the Hough transform parameters
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 18 # minimum number of pixels making up a line
    max_line_gap = 30 # maximum gap in pixels between connectable line segments
    lines,hough_line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    #combine Hough lines with original image
    combined_image = weighted_img(hough_line_image, image)
    
    return combined_image

In [2]:
image_list = os.listdir("test_images/")
for image_name in image_list:
    full_image_path = 'test_images/' + image_name
    find_lane_lines_raw(full_image_path)


Finding raw lane lines for:  test_images/solidWhiteCurve.jpg
Writing processed image:  test_images_output/raw-solidWhiteCurve.jpg 

Finding raw lane lines for:  test_images/solidWhiteRight.jpg
Writing processed image:  test_images_output/raw-solidWhiteRight.jpg 

Finding raw lane lines for:  test_images/solidYellowCurve.jpg
Writing processed image:  test_images_output/raw-solidYellowCurve.jpg 

Finding raw lane lines for:  test_images/solidYellowCurve2.jpg
Writing processed image:  test_images_output/raw-solidYellowCurve2.jpg 

Finding raw lane lines for:  test_images/solidYellowLeft.jpg
Writing processed image:  test_images_output/raw-solidYellowLeft.jpg 

Finding raw lane lines for:  test_images/whiteCarLaneSwitch.jpg
Writing processed image:  test_images_output/raw-whiteCarLaneSwitch.jpg 



In [3]:
image_list = os.listdir("test_images/")
for image_name in image_list:
    full_image_path = 'test_images/' + image_name
    find_lane_lines(full_image_path)

Finding lane lines for:  test_images/solidWhiteCurve.jpg
Writing processed image:  test_images_output/solidWhiteCurve.jpg 

Finding lane lines for:  test_images/solidWhiteRight.jpg
Writing processed image:  test_images_output/solidWhiteRight.jpg 

Finding lane lines for:  test_images/solidYellowCurve.jpg
Writing processed image:  test_images_output/solidYellowCurve.jpg 

Finding lane lines for:  test_images/solidYellowCurve2.jpg
Writing processed image:  test_images_output/solidYellowCurve2.jpg 

Finding lane lines for:  test_images/solidYellowLeft.jpg
Writing processed image:  test_images_output/solidYellowLeft.jpg 

Finding lane lines for:  test_images/whiteCarLaneSwitch.jpg
Writing processed image:  test_images_output/whiteCarLaneSwitch.jpg 



In [4]:
white_output_raw = 'test_videos_output/raw-solidWhiteRight.mp4'
clip1_raw = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip_raw = clip1_raw.fl_image(process_image_raw_lines) 
%time white_clip_raw.write_videofile(white_output_raw, audio=False)

[MoviePy] >>>> Building video test_videos_output/raw-solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/raw-solidWhiteRight.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 221/222 [00:07<00:00, 24.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/raw-solidWhiteRight.mp4 

Wall time: 9.07 s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_output_raw))

In [6]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 221/222 [00:07<00:00, 27.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 9.03 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_output))

In [8]:
yellow_output_raw = 'test_videos_output/raw-solidYellowLeft.mp4'
clip2_raw = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip_raw = clip2_raw.fl_image(process_image_raw_lines)
%time yellow_clip_raw.write_videofile(yellow_output_raw, audio=False)

[MoviePy] >>>> Building video test_videos_output/raw-solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/raw-solidYellowLeft.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:26<00:00, 24.73it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/raw-solidYellowLeft.mp4 

Wall time: 28 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_output_raw))

In [10]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:25<00:00, 27.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 26.3 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_output))

In [12]:
challenge_output_raw = 'test_videos_output/raw-challenge.mp4'
clip3_raw = VideoFileClip('test_videos/challenge.mp4')
challenge_clip_raw = clip3_raw.fl_image(process_image_raw_lines)
%time challenge_clip_raw.write_videofile(challenge_output_raw, audio=False)

[MoviePy] >>>> Building video test_videos_output/raw-challenge.mp4
[MoviePy] Writing video test_videos_output/raw-challenge.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:13<00:00, 19.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/raw-challenge.mp4 

Wall time: 14.4 s


In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(challenge_output_raw))

In [14]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:13<00:00, 19.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 15.3 s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(challenge_output))